In [ ]:
import nltk
# Download NLTK data (if not already downloaded)
nltk.download("stopwords")
nltk.download("punkt")

In [ ]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1")
df = df.iloc[0:15000,:]
df.to_csv("text_training_data.csv")
df.shape

In [ ]:
import pandas as pd
# Load the dataset
# Replace 'your_dataset.csv' with the actual path to your dataset file
# The dataset should have columns: polarity, id, date, query, user, and text
# 0 = negative, 2 = neutral, 4 = positive
dataset_path = "text_training_data.csv"
df = pd.read_csv(dataset_path, encoding="latin-1", names=["polarity", "id", "date", "query", "user", "text"])

# Drop unnecessary columns
df = df[["polarity", "text"]]

# Shuffle the dataset
df = df.sample(frac=1, random_state=42)

# Define a function to preprocess the text
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and convert to lowercase
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return " ".join(filtered_words)

# Preprocess the dataset
df["text"] = df["text"].apply(preprocess_text)

# Split the dataset into training and testing sets
X = df["text"].tolist()
y = df["polarity"].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Transform text data into TF-IDF features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:

# Train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel="linear")
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment labels for the test set
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)


In [ ]:
import time
import sklearn.metrics as sm
from sklearn.utils import all_estimators
import pickle
estimators = all_estimators(type_filter='classifier')
model_name=[]
model_precision=[]
model_accuracy=[]
model_training_time=[]
classification_rep = []
model_path = "Text.model.save"
for name, get_model in estimators:
    try:
        model = get_model()
        # Train the classifier and measure time
        start_time = time.time()
        model.fit(X_train_tfidf,y_train)
        y_pred=model.predict(X_test_tfidf)
        end_time = time.time()
        classification_rep.append(classification_report(y_test, y_pred))
        model_precision.append(sm.precision_score(y_test, y_pred))
        model_accuracy.append(sm.accuracy_score(y_test, y_pred))
        model_name.append(name)
        model_training_time.append(end_time - start_time)
        # Save model with pickle model
        with open(model_path + f"/{name}.pkl",'wb') as model_file:
            pickle.dump(model, model_file)
    except Exception as e:
        print('Unable to import', name)
        print(e)

In [ ]:
results=pd.DataFrame({"Model Name":model_name, "Model accuracy":model_accuracy, "Model training time":model_training_time , "Model precision":model_precision})
results